In [265]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [266]:

import scipy.spatial as ss
from scipy.special import digamma
from math import log
import numpy.random as nr
import numpy as np
import random
import time
import pandas as pd
from sklearn import svm

from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier
import math

In [267]:

def lcsi(X, y, **kwargs):
    n_samples, n_features = X.shape
    # index of selected features, initialized to be empty
    F = []
    # Objective function value for selected features
    J_CMI = []
    # Mutual information between feature and response
    MIfy = []
    # indicate whether the user specifies the number of features
    is_n_selected_features_specified = False
    # initialize the parameters
    if 'beta' in kwargs.keys():
        beta = kwargs['beta']
    if 'gamma' in kwargs.keys():
        gamma = kwargs['gamma']
    if 'n_selected_features' in kwargs.keys():
        n_selected_features = kwargs['n_selected_features']
        is_n_selected_features_specified = True

    # select the feature whose j_cmi is the largest
    # t1 stores I(f;y) for each feature f
    t1 = np.zeros(n_features)
   
    # t2 stores sum_j(I(fj;f)) for each feature f
    t2 = np.zeros(n_features)
    # t3 stores sum_j(I(fj;f|y)) for each feature f
    t3 = np.zeros(n_features)
    for i in range(n_features):
        f = X[:, i]
        t1[i] = midd(f, y)
    #print(t1)
    # make sure that j_cmi is positive at the very beginning
    j_cmi = 1

    while True:
        if len(F) == 0:
            # select the feature whose mutual information is the largest
            idx = np.argmax(t1)
            F.append(idx)
            J_CMI.append(t1[idx])
            MIfy.append(t1[idx])
            f_select = X[:, idx]

        if is_n_selected_features_specified:
            if len(F) == n_selected_features:
                break
        else:
            if j_cmi < 0:
                break

        # we assign an extreme small value to j_cmi to ensure it is smaller than all possible values of j_cmi
        j_cmi = -1E30
        if 'function_name' in kwargs.keys():
            if kwargs['function_name'] == 'MRMR':
                beta = 1.0 / len(F)
            elif kwargs['function_name'] == 'JMI':
                beta = 1.0 / len(F)
                gamma = 1.0 / len(F)
        for i in range(n_features):
            if i not in F:
                f = X[:, i]
                t2[i] += midd(f_select, f)
                t3[i] += cmidd(f_select, f, y)
                # calculate j_cmi for feature i (not in F)
                t = t1[i] - beta*t2[i] + gamma*t3[i]
                # record the largest j_cmi and the corresponding feature index
                if t > j_cmi:
                    j_cmi = t
                    idx = i
        F.append(idx)
        J_CMI.append(j_cmi)
        MIfy.append(t1[idx])
        f_select = X[:, idx]

    return np.array(F), np.array(J_CMI), np.array(MIfy)

In [268]:

# continuous estimators

def entropy(x, k=3, base=2):

    assert k <= len(x)-1, "Set k smaller than num. samples - 1"
    d = len(x[0])
    N = len(x)
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    tree = ss.cKDTree(x)
    nn = [tree.query(point, k+1, p=float('inf'))[0][k] for point in x]
    const = digamma(N)-digamma(k) + d*log(2)
    return (const + d*np.mean(map(log, nn)))/log(base)

def mi(x, y, k=3, base=2):

    assert len(x) == len(y), "Lists should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    y = [list(p + intens * nr.rand(len(y[0]))) for p in y]
    points = zip2(x, y)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = ss.cKDTree(points)
    dvec = [tree.query(point, k+1, p=float('inf'))[0][k] for point in points]
    a, b, c, d = avgdigamma(x, dvec), avgdigamma(y, dvec), digamma(k), digamma(len(x))
    return (-a-b+c+d)/log(base)

def cmi(x, y, z, k=3, base=2):
    """
    Mutual information of x and y, conditioned on z; x, y, z should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert len(x) == len(y), "Lists should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    y = [list(p + intens * nr.rand(len(y[0]))) for p in y]
    z = [list(p + intens * nr.rand(len(z[0]))) for p in z]
    points = zip2(x, y, z)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = ss.cKDTree(points)
    dvec = [tree.query(point, k+1, p=float('inf'))[0][k] for point in points]
    a, b, c, d = avgdigamma(zip2(x, z), dvec), avgdigamma(zip2(y, z), dvec), avgdigamma(z, dvec), digamma(k)
    return (-a-b+c+d)/log(base)

def kldiv(x, xp, k=3, base=2):
    """
    KL Divergence between p and q for x~p(x), xp~q(x); x, xp should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    assert k <= len(xp) - 1, "Set k smaller than num. samples - 1"
    assert len(x[0]) == len(xp[0]), "Two distributions must have same dim."
    d = len(x[0])
    n = len(x)
    m = len(xp)
    const = log(m) - log(n-1)
    tree = ss.cKDTree(x)
    treep = ss.cKDTree(xp)
    nn = [tree.query(point, k+1, p=float('inf'))[0][k] for point in x]
    nnp = [treep.query(point, k, p=float('inf'))[0][k-1] for point in x]
    return (const + d*np.mean(map(log, nnp))-d*np.mean(map(log, nn)))/log(base)

# Discrete estimators
def entropyd(sx, base=2):
    """
    Discrete entropy estimator given a list of samples which can be any hashable object
    """

    return entropyfromprobs(hist(sx), base=base)


def midd(x, y):
    """
    Discrete mutual information estimator given a list of samples which can be any hashable object
    """

    return -entropyd(list(zip(x, y)))+entropyd(x)+entropyd(y)


def cmidd(x, y, z):
    """
    Discrete mutual information estimator given a list of samples which can be any hashable object
    """

    return entropyd(list(zip(y, z)))+entropyd(list(zip(x, z)))-entropyd(list(zip(x, y, z)))-entropyd(z)


def hist(sx):
    # Histogram from list of samples
    d = dict()
    for s in sx:
        d[s] = d.get(s, 0) + 1
    return map(lambda z: float(z)/len(sx), d.values())


def entropyfromprobs(probs, base=2):
    # Turn a normalized list of probabilities of discrete outcomes into entropy (base 2)
    return -sum(map(elog, probs))/log(base)


def elog(x):
    # for entropy, 0 log 0 = 0. but we get an error for putting log 0
    if x <= 0. or x >= 1.:
        return 0
    else:
        return x*log(x)


# Mixed estimators
def micd(x, y, k=3, base=2, warning=True):
    """ If x is continuous and y is discrete, compute mutual information
    """

    overallentropy = entropy(x, k, base)
    n = len(y)
    word_dict = dict()
    for sample in y:
        word_dict[sample] = word_dict.get(sample, 0) + 1./n
    yvals = list(set(word_dict.keys()))

    mi = overallentropy
    for yval in yvals:
        xgiveny = [x[i] for i in range(n) if y[i] == yval]
        if k <= len(xgiveny) - 1:
            mi -= word_dict[yval]*entropy(xgiveny, k, base)
        else:
            if warning:
                print("Warning, after conditioning, on y={0} insufficient data. Assuming maximal entropy in this case.".format(yval))
            mi -= word_dict[yval]*overallentropy
    return mi  # units already applied


# Utility functions
def vectorize(scalarlist):
    """
    Turn a list of scalars into a list of one-d vectors
    """

    return [(x,) for x in scalarlist]


def shuffle_test(measure, x, y, z=False, ns=200, ci=0.95, **kwargs):
    """
    Shuffle test
    Repeatedly shuffle the x-values and then estimate measure(x,y,[z]).
    Returns the mean and conf. interval ('ci=0.95' default) over 'ns' runs, 'measure' could me mi,cmi,
    e.g. Keyword arguments can be passed. Mutual information and CMI should have a mean near zero.
    """

    xp = x[:]   # A copy that we can shuffle
    outputs = []
    for i in range(ns):
        random.shuffle(xp)
        if z:
            outputs.append(measure(xp, y, z, **kwargs))
        else:
            outputs.append(measure(xp, y, **kwargs))
    outputs.sort()
    return np.mean(outputs), (outputs[int((1.-ci)/2*ns)], outputs[int((1.+ci)/2*ns)])


# Internal functions
def avgdigamma(points, dvec):
    # This part finds number of neighbors in some radius in the marginal space
    # returns expectation value of <psi(nx)>
    N = len(points)
    tree = ss.cKDTree(points)
    avg = 0.
    for i in range(N):
        dist = dvec[i]
        # subtlety, we don't include the boundary point,
        # but we are implicitly adding 1 to kraskov def bc center point is included
        num_points = len(tree.query_ball_point(points[i], dist-1e-15, p=float('inf')))
        avg += digamma(num_points)/N
    return avg


def zip2(*args):
    # zip2(x,y) takes the lists of vectors and makes it a list of vectors in a joint space
    # E.g. zip2([[1],[2],[3]],[[4],[5],[6]]) = [[1,4],[2,5],[3,6]]
    return [sum(sublist, []) for sublist in zip(*args)]


In [269]:
def mim(X, y, **kwargs):
    if 'n_selected_features' in kwargs.keys():
        n_selected_features = kwargs['n_selected_features']
        F, J_CMI, MIfy = lcsi(X, y, beta=0, gamma=0, n_selected_features=n_selected_features)
    else:
        F, J_CMI, MIfy = lcsi(X, y, beta=0, gamma=0)
    return F, J_CMI, MIfy

In [270]:
from google.colab import drive 
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [271]:
# load data

dataset = 'CreditRisk.csv'
link = '/content/drive/MyDrive/FeatureSelection/dataset/' + dataset
df=pd.read_csv(link)
nRow, nCol = df.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,45,53) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [272]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_col

In [273]:
#n_samples = 11000;
#df = df.groupby('y', group_keys=False).apply(lambda x: x.sample(frac=n_samples/nRow))
df.shape

(855969, 73)

In [274]:
#df = df.dropna(axis='rows')
df = df.fillna(0)
cat_columns = df.select_dtypes(['object']).columns
if not cat_columns.empty:
  df[cat_columns] = df[cat_columns].apply(lambda x: pd.factorize(x)[0])
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,y
0,1077501,1296599,5000,5000,4975.0,0,10.65,162.87,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1077430,1314167,2500,2500,2500.0,1,15.27,59.83,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1077175,1313524,2400,2400,2400.0,0,15.96,84.33,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1076863,1277178,10000,10000,10000.0,0,13.49,339.31,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1075358,1311748,3000,3000,3000.0,1,12.69,67.79,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [275]:
X = df.drop(columns=['id','y']).to_numpy()

features_label = df.columns.difference(['y']).to_numpy()
y = df['y'].to_numpy()
n_samples, n_features = X.shape



In [ ]:
start_time=time.time()
# perform evaluation on classification task
num_fea = n_features    # number of selected features

idx,_,_ = mim(X, y, n_selected_features=num_fea)
#print(features_label)
#print(score)
#print(idx)
# obtain the dataset on the selected features
features = X[:, idx[0:num_fea]]


In [ ]:
idx_top10 = idx[0:int(n_features/10)]
print("--------------------------------------")
print("index of top  10 : %s" %idx_top10)
print("Top 10 feature: %s " %features_label[idx_top10])

In [ ]:
idx_top15 = idx[0:int(15*n_features/100)]
print("--------------------------------------")
print("index of top  15 : %s" %idx_top15)
print("Top 15 feature: %s " %features_label[idx_top15])

In [ ]:
idx_top25 = idx[0:int(n_features/4)]
print("--------------------------------------")
print("index of top  25: %s" %idx_top25)
print("Top 25 feature: %s " %features_label[idx_top25])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
X_selected = X[:, idx_top10]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)


In [ ]:
# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

y_predict
acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', accuracy_score(y_test,y_predict))
print('ROC AUC =', roc_auc_score(y_test, y_predict ))
print('F1 =', f1_score(y_test,y_predict))

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'MIM','name':[dataset], 'samples': [n_samples], 'top': '10','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)


In [ ]:
X_selected = X[:, idx_top15]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)


In [ ]:
# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', acc)
print('ROC AUC =', auc)
print('F1 =', f1)

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'MIM','name':[dataset], 'samples': [n_samples], 'top': '15','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)


In [ ]:
X_selected = X[:, idx_top25]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)


In [ ]:
# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

y_predict
acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', accuracy_score(y_test,y_predict))
print('ROC AUC =', roc_auc_score(y_test, y_predict ))
print('F1 =', f1_score(y_test,y_predict))

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'MIM','name':[dataset], 'samples': [n_samples], 'top': '25','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)
